# Communicate with your digital twin: Cloud harvester
This is a enterprise application which allows to access to your digital twin which is launched in the S³I Repository as cloud DT. This Notebook acts as an application in terms of the S³I and a client in terms of OAuth2 authentication.
Just go to the **Cell** drop-down menu and click on the **Run All** button.

In [ ]:
import s3i
import json
import getpass
import os 
from tools import print_with_timestamp, yes, no
import ml
#from ml.tools import find_broker_endpoint, make_thing_config, load_config
#from ml.dt_factory import create_thing
#from ml.app_logger import setup_logger

In [ ]:
%%html
<img src="harvester_images/folie_1_3.png", width=1000, height=1000>

## Configure the notebook
In order to assign this notebook to your private application it needs a client id and the respective secret. In the following input fields, you can enter the id and the secret of your application.

In [ ]:
print_with_timestamp("Assign a client to this notebook. (The id of your application)")
app_id = input('[S3I]: Please enter your application id:').strip('," ')
app_secret = getpass.getpass('[S3I]: Please enter your application secret:').strip('," ')
print_with_timestamp("App id and secret are set.")

Next, you have to enter your username and password to authenticate yourself in S³I. 

In [ ]:
print_with_timestamp("This is a enterprise app, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your username and password are set.")

## Build and load
Analogously to the harvester, a application can also be created and launched using fml40 reference implementation library. All defined roles, functionalities and values etc. are respectively implemented according to the fml40 language. Next, a JSON file for the app is created and loaded into the notebook.

In [ ]:
app_name = "my_app_cloud"
config_path = os.path.abspath(os.path.join("", "configs"))
config_file_name = ml.make_thing_config(thing_id=app_id, name=app_name, roles=[{"class": "ml40::App"}],
                                     config_path=config_path)
app_model = ml.load_config(config_filepath=os.path.join(config_path, config_file_name))
app = ml.create_thing(
                    model=app_model, grant_type="password", secret=app_secret,
                    username=username, password=password, 
                    is_broker=False, is_repo=False
                    )

## Setup a loger for the App

In [ ]:
ml.setup_logger(app_name)

## Run the application 
The created application is going to be launched in the following step. When the run_forever function is called, the application is started, and meanwhile its listener is also started to receive the messages sent to it via S³I Broker. Additionally, you can specific the receiver, with whom it will communicate. 

In [ ]:
app.run_forever()
config_path = os.path.abspath(os.path.join("", "configs"))
harvester_name = input("[S³I]: Please enter the name of your harvester: (my_cloud_harvester) ")
cred_filepath = os.path.join(config_path, "{}_cred.json".format(harvester_name))
with open(cred_filepath) as file:
    cred = json.load(file)
receiver = cred.get("identifier")
app_endpoint = ml.find_broker_endpoint(app.dir, app_id)
receiver_endpoint = ml.find_broker_endpoint(app.dir, thing_id=receiver)

## Get the directory entry of the cloud harvester

In [ ]:
%%html
<img src="harvester_images/dir.png", width=300, height=300>

In [ ]:
print_with_timestamp("The dir entry of the cloud harvester: " + json.dumps(app.dir.queryThingIDBased(thingID=receiver), indent=2))

## Get the repository entry of the cloud harvester

In [ ]:
%%html
<img src="harvester_images/repo.png", width=500, height=500>

In [ ]:
print_with_timestamp("The repo entry of the cloud harvester" + json.dumps(app.repo.queryThingIDBased(thingID=receiver), indent=2))